# Setup

Der Code unterhalb braucht nur gensim, kann mit `pip install gensim` installiert werden. 

In [1]:
import os
import xml.etree.ElementTree as ET
from gensim.models import Word2Vec
from gensim.models import FastText
import re
from pathlib import Path
from typing import List
import random

# Word2Vec trainieren

Der Code unterhalb ist nur relevant, wenn du das Modell neu trainieren willst - ein fertiges Modell ist allerdings schon geuploaded. 

Der Code zum Testen / Anwenden ist weiter unten. 

Ich habe die Einstellungen nochmal ein bisschen verändert: 
100 ist vollkommen ausreichend als dimension, und ein größeres window erreicht, dass mehr Information rausgeholt wird.  

In [ ]:
# Train Model
model = FastText(
    corpus_file="corpus_cleaned.txt",
    vector_size=300,  # Dimensionality of the word embeddings
    window=10,         # Max distance between current and predicted word
    min_count=3,      # Ignores all words with total frequency < 3
    workers=8,        # Number of threads to run in parallel
    epochs=5,        # Number of training epochs
    sg=1
)

# 4) Save the trained model for later use
model.save("word2vec.model")

In [4]:
# Assume `model` is your trained Word2Vec model
vocabulary = model.wv.key_to_index

# Print the vocabulary (all words)
with open("vocabulary.txt", 'w') as fw: 
    fw.writelines('\n'.join(list(vocabulary.keys())))

# Modell anwenden

Wenn in fastText-Modell verwendet wird (ist beim aktuellen word2vec-Modell und dem zugehörigen Code oberhalb der Fall), können beliebige Sequenzen und Wörter getestet werden, unabhängig davon, ob diese in den Trainingsdaten waren. In diesem Fall wird die Eingabe in einzelne, kleinere Buchstabenfolgen zerlegt, die sich in den Trainingsdaten finden. Trotzdem ist das Ergebnis auf Wörtern aus dem Vokabular (kann mit dem Code direkt oberhalb in eine Datei geschrieben werden) vermutlich ab besten. 

Wenn ein normales Word2Vec-Modell verwendet wird (nicht empfohlen, schlechtere Ergebnisse), können nur Wörter aus dem Vokabular eingegeben werden.

In [17]:
def find_contexts(
    file_path: str | Path,
    target_word: str,
    window: int = 3,
    max_examples: int = 5
) -> None:
    """Zählt Vorkommen von `target_word` und gibt Kontexte (±window) aus."""
    text = Path(file_path).read_text(encoding="utf-8", errors="ignore")

    sentences = text.split('\n')
    matches = []
    target_word = target_word.lower()
    for s_idx, sent in enumerate(sentences): 
        sent_lower = sent.lower()
        if target_word in sent_lower: # matches
            for i, char in enumerate(sent_lower): 
                if sent_lower[i:i + len(target_word)] == target_word: 
                    matches.append((i, sent))


    sampled_matches = random.sample(matches, k=min(5, len(matches)))
    print(f"Found {len(matches)} matches for '{target_word}' in {file_path}.")
    for n, match_el in enumerate(sampled_matches):
        idx = match_el[0]
        sent = match_el[1]
        start = max(idx - window, 0)
        end = min(idx + window + len(target_word) +1, len(sent))
        
        out_text = sent[idx:idx+len(target_word)].upper()
        if start < idx: 
            out_text = sent[start:idx] + out_text
        if idx < end: 
            out_text = out_text + sent[idx+len(target_word):end]
        if start > 0: 
            out_text = '... ' + out_text
        if end < len(sent): 
            out_text = out_text + ' ...'
        print(out_text)

In [4]:
# Load Model
model = FastText.load("word2vec.model")

In [ ]:
target_word = 'juifs'
similar_words = model.wv.most_similar(target_word, topn=25)
print(similar_words, '\n')
find_contexts("corpus_raw.txt", target_word, window =70)

[('ethnomasochisme', 0.9348476529121399), ('masochiste', 0.919653058052063), ('djihadistes', 0.8303912878036499), ('masochisme', 0.817060112953186), ('islamo_gauchistes', 0.8088613152503967), ('#islamogauchisme', 0.8024135828018188), ('franciliens', 0.7978147268295288), ('matérialistes', 0.7937813997268677), ('gpa', 0.7932493090629578), ('israéliens', 0.7932366132736206), ('universalistes', 0.7915453314781189), ('ethno', 0.7905784249305725), ('indigénistes', 0.7894279360771179), ('djihadiste', 0.7891472578048706), ('minorités_ethniques', 0.7823185920715332), ('mouvements_nationalistes', 0.7803925275802612), ('terrorisme_islamiste', 0.7795858383178711), ('gouvernements_successifs', 0.7766215801239014), ('racialistes', 0.7760908007621765), ('anasse_kazib', 0.7757160067558289), ('dirigeants', 0.775704562664032), ('parti_communiste', 0.7745497822761536), ('socialistes', 0.772439181804657), ('lacrymogène', 0.766057014465332), ('terroristes', 0.7633559703826904)] 

Found 5 matches for 'ethno

In [110]:
def char_overlap(target_word, neighbours:list): 
    overlaps = []
    lens = []
    for neighbour in neighbours: 
        if len(target_word) > len(neighbour): 
            longer_word = target_word
            shorter_word = neighbour
        else: 
            longer_word = neighbour
            shorter_word = target_word
        lens.append(len(shorter_word))
        n_gram_len = len(shorter_word)
        found_overlap = False
        print(neighbour)
        while n_gram_len > 0 and not found_overlap: 
            for starting_pos in range(len(shorter_word) - n_gram_len + 1): 
                #print(starting_pos)
                n_gram = shorter_word[starting_pos:starting_pos+n_gram_len]
                #print(n_gram)
                if n_gram in longer_word: 
                    #print('yes')
                    overlaps.append(n_gram_len)
                    found_overlap = True
                    break
            n_gram_len -= 1
        if n_gram_len == 0 and not found_overlap: 
            overlaps.append(0)
    
    print(overlaps)
    print(lens)
    sum = 0
    for i, overlap in enumerate(overlaps): 
        # Prints fraction of *maximum possible overlap* for those words 
        sum += overlap/lens[i]
    print(sum/len(overlaps))